# IMDB: recursive neural networks

## Data preprocessing

### Required imports

In [7]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import numpy as np
from sklearn.model_selection import train_test_split

### Processing

Load the training and test data.

In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data()

Since the review vary in length, and we prefer to limit the computation time, we will base the classification on the first 100 features of each input sequence.

In [3]:
feature_length = 100
x_train = sequence.pad_sequences(x_train, maxlen=feature_length)
x_test = sequence.pad_sequences(x_test, maxlen=feature_length)

Again, to limit computation time, we restrict the number of features to 5000.  This means we map all features with an index of 20000 or more to 0.

In [4]:
nr_features = 5_000
x_train[x_train > nr_features - 1] = 0
x_test[x_test > nr_features - 1] = 0

Now the training and test input are 2D arrays. We split the training set into a subset for actual training, and one for validation.  First we seed the random number generator to ensure reproducibility. In this case, we will use part of the 25000 test examples as valiation data.

In [8]:
np.random.seed(1234)

In [9]:
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test)

## Model defintion

### Required imports

In [14]:
from keras.layers import Activation, Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import GRU
from keras.models import Sequential
from keras.optimizers import Adam

Again, to limit training times, we restrict ourselfs to using a limited number of features.

In [24]:
model = Sequential()
model.add(Embedding(nr_features, 64, mask_zero=True,
                    input_length=feature_length))
model.add(GRU(64))
model.add(Dropout(0.7))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy', optimizer=Adam(),
              metrics=['accuracy'])

##    Training

In [26]:
history = model.fit(x_train, y_train, batch_size=64, epochs=10,
                    validation_data=(x_val, y_val))

Train on 25000 samples, validate on 6250 samples
Epoch 1/10
25000/25000 [==============================] - 85s 3ms/step - loss: 0.4612 - acc: 0.7732 - val_loss: 0.3691 - val_acc: 0.8411
Epoch 2/10
25000/25000 [==============================] - 75s 3ms/step - loss: 0.3118 - acc: 0.8734 - val_loss: 0.3512 - val_acc: 0.8506
Epoch 3/10
25000/25000 [==============================] - 80s 3ms/step - loss: 0.2771 - acc: 0.8901 - val_loss: 0.3691 - val_acc: 0.8443
Epoch 4/10
25000/25000 [==============================] - 173s 7ms/step - loss: 0.2358 - acc: 0.9085 - val_loss: 0.3753 - val_acc: 0.8390
Epoch 5/10
25000/25000 [==============================] - 96s 4ms/step - loss: 0.1977 - acc: 0.9267 - val_loss: 0.4224 - val_acc: 0.8402
Epoch 6/10
25000/25000 [==============================] - 108s 4ms/step - loss: 0.1641 - acc: 0.9414 - val_loss: 0.5014 - val_acc: 0.8275
Epoch 7/10
25000/25000 [==============================] - 98s 4ms/step - loss: 0.1425 - acc: 0.9496 - val_loss: 0.5060 - val_ac

The training accuracy is much better than the validation accurcy, so the model is likely heavily overtrained.

In [27]:
model.evaluate(x_test, y_test)

18750/18750 [==============================] - 31s 2ms/step


[0.5877906380399068, 0.8167999999809266]